In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Paths and stuff
import os
import sys

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [2]:
KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
time_ad = np.arange(12, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1

In [3]:
def prior_inv_sqrt_action(field, gamma, delta, delta_x = 40.0, delta_y = 40.0):

    field_new = delta*field.copy()

    field_new[0, 0]       = field_new[0, 0]       - gamma*(field[0, 1]         / delta_x**2 + field[1, 0]         / delta_y**2)
    field_new[JMAX, 0]    = field_new[JMAX, 0]    - gamma*(field[JMAX, 1]      / delta_x**2 + field[JMAX-1, 0]      / delta_y**2)
    field_new[0, IMAX]    = field_new[0, IMAX]    - gamma*(field[0, IMAX-1]    / delta_x**2 + field[1, IMAX]      / delta_y**2)
    field_new[JMAX, IMAX] = field_new[JMAX, IMAX] - gamma*(field[JMAX, IMAX-1] / delta_x**2 + field[JMAX-1, IMAX] / delta_y**2)

    field_new[1:JMAX, 0]    = field_new[1:JMAX, 0]    - gamma*((field[0:JMAX-1, 0]    - 2*field[1:JMAX, 0]    + field[2:, 0])    / delta_y**2 + (field[1:JMAX, 1]      - field[1:JMAX, 0])    / delta_x**2)
    field_new[1:JMAX, IMAX] = field_new[1:JMAX, IMAX] - gamma*((field[0:JMAX-1, IMAX] - 2*field[1:JMAX, IMAX] + field[2:, IMAX]) / delta_y**2 + (field[1:JMAX, IMAX-1] - field[1:JMAX, IMAX]) / delta_x**2)

    field_new[0, 1:IMAX]    = field_new[0, 1:IMAX]    - gamma*((field[1, 1:IMAX]      - field[0, 1:IMAX])    / delta_y**2 + (field[0, 0:IMAX-1]    - 2*field[0, 1:IMAX]    + field[0, 2:])    / delta_x**2)
    field_new[JMAX, 1:IMAX] = field_new[JMAX, 1:IMAX] - gamma*((field[JMAX-1, 1:IMAX] - field[JMAX, 1:IMAX]) / delta_y**2 + (field[JMAX, 0:IMAX-1] - 2*field[JMAX, 1:IMAX] + field[JMAX, 2:]) / delta_x**2)
    
    for j in range(1, JMAX):
        for i in range(1, IMAX):
            field_new[j, i] = field_new[j, i] - gamma*(field[j, i-1] - 2*field[j, i] + field[j, i+1]) / delta_x**2
            field_new[j, i] = field_new[j, i] - gamma*(field[j-1, i] - 2*field[j, i] + field[j+1, i]) / delta_y**2
    
    return field_new

def prior_sqrt_action(field, omega, gamma, delta, delta_x = 40.0, delta_y = 40.0):

    out_old = np.copy(field)
    out_new = np.copy(field)

    iterations = 0
    while True:
        
        iterations = iterations + 1
        
        for j in range(JMAX+1):
            for i in range(IMAX+1):

                if j == 0 and i == 0:
                    diagonal = delta
                    bracket = field[0, 0] + gamma*(out_old[0, 1] / delta_x**2 + out_old[1, 0] / delta_y**2)
                elif j == JMAX and i == 0:
                    diagonal = delta
                    bracket = field[JMAX, 0] + gamma*(out_old[JMAX, 1] / delta_x**2 + out_new[JMAX-1, 0] / delta_y**2)
                elif j == 0 and i == IMAX:
                    diagonal = delta
                    bracket = field[0, IMAX] + gamma*(out_new[0, IMAX-1] / delta_x**2 + out_old[1, IMAX] / delta_y**2)
                elif j == JMAX and i == IMAX:
                    diagonal = delta
                    bracket = field[JMAX, IMAX] + gamma*(out_new[JMAX, IMAX-1] / delta_x**2 + out_new[JMAX-1, IMAX] / delta_y**2)
                elif i == 0:
                    diagonal = delta + gamma*(1/delta_x**2 + 2/delta_y**2)
                    bracket = field[j, 0] + gamma*((out_new[j-1, 0] + out_old[j+1, 0]) / delta_y**2 + out_old[j, 1] / delta_x**2)
                elif i == IMAX:
                    diagonal = delta + gamma*(1/delta_x**2 + 2/delta_y**2)
                    bracket = field[j, IMAX] + gamma*((out_new[j-1, IMAX] + out_old[j+1, IMAX]) / delta_y**2 + out_new[j, IMAX-1] / delta_x**2)
                elif j == 0:
                    diagonal = delta + gamma*(2/delta_x**2 + 1/delta_y**2)
                    bracket = field[0, i] + gamma*(out_old[1, i] / delta_y**2 + (out_new[0, i-1] + out_old[0, i+1]) / delta_x**2)
                elif j == JMAX:
                    diagonal = delta + gamma*(1/delta_x**2 + 2/delta_y**2)
                    bracket = field[JMAX, i] + gamma*(out_new[JMAX-1, i] / delta_y**2 + (out_new[JMAX, i-1] + out_old[JMAX, i+1]) / delta_x**2)
                else:
                    diagonal = delta + 2*gamma*(1/delta_x**2 + 1/delta_y**2)
                    bracket = field[j, i] + gamma*((out_new[j-1, i] + out_old[j+1, i]) / delta_y**2 + (out_new[j, i-1] + out_old[j, i+1]) / delta_x**2)

                out_new[j, i] = (1 - omega) * out_old[j, i] + omega / diagonal * bracket
                
        out_old = out_new.copy()

        if iterations == 100:
            break

    return out_new

delta = 0.1
gamma = 1

random = np.random.randn(JMAX+1, IMAX+1)
random_new = prior_inv_sqrt_action(random, gamma, delta)
random_recovered = prior_sqrt_action(random_new, 1.5, gamma, delta)
np.mean(random), np.std(random), np.mean(random_new), np.std(random_new), np.mean(random_recovered), np.std(random_recovered)

(0.03783680594221125,
 0.9873575847457544,
 0.0037841190080716396,
 0.10114961230463275,
 0.03783680594221126,
 0.9873575847457544)

In [4]:
random = log_c_slide_init = np.log10(8.5)*np.ones((JMAX+1, IMAX+1))
random_new = prior_inv_sqrt_action(random, gamma, delta)
random_recovered = prior_sqrt_action(random_new, 1.5, gamma, delta)
np.mean(random), np.std(random), np.mean(random_new), np.std(random_new), np.mean(random_recovered), np.std(random_recovered)

(0.9294189257142927,
 0.0,
 0.09294041213351001,
 4.1445652114596894e-05,
 0.9294189257142927,
 1.1116368717750646e-16)